# 10장 딥러닝 모델

In [ ]:
# 필요 라이브러리 선언
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# PDF 출력
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

## 10.7 프로그램 구현(1)

### 데이터 읽기

In [ ]:
# 데이터 읽기
import os
import urllib.request

mnist_file = 'mnist-original.mat'
mnist_path = 'mldata'
mnist_url = 'https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat'

# 파일 존재 확인
mnist_fullpath = os.path.join('.', mnist_path, mnist_file)
if not os.path.isfile(mnist_fullpath):
    # 데이터 다운로드
    mldir = os.path.join('.', 'mldata')
    os.makedirs(mldir, exist_ok=True)
    print("donwload %s started." % mnist_file)
    urllib.request.urlretrieve(mnist_url, mnist_fullpath)
    print("donwload %s finished." % mnist_file)

from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home='.')

In [ ]:
x_org, y_org = mnist.data, mnist.target

### 입력 데이터의 가공

In [ ]:
# 입력 데이터의 가공

# step1 데이터 정규화 값의 범위를 [0, 1]로 제한
x_norm = x_org / 255.0

# 앞에 더미 데이터 변수(1)를 추가
x_all = np.insert(x_norm, 0, 1, axis=1)

print('더미 변수 추가 후', x_all.shape)

In [ ]:
# step 2 y를 One-hot-Vector로

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_all_one = ohe.fit_transform(np.c_[y_org])
print('원핫 인코딩', y_all_one.shape)

In [ ]:
# step 3 학습 데이터와 검증 데이터로 분할

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, y_train_one, y_test_one = train_test_split(
    x_all, y_org, y_all_one, train_size=60000, test_size=10000, shuffle=False)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, 
    y_train_one.shape, y_test_one.shape)

In [ ]:
# 데이터 내용 확인

N = 20
np.random.seed(123)
indexes = np.random.choice(y_test.shape[0], N, replace=False)
x_selected = x_test[indexes,1:]
y_selected = y_test[indexes]
plt.figure(figsize=(10, 3))
for i in range(N):
    ax = plt.subplot(2, N/2, i + 1)
    plt.imshow(x_selected[i].reshape(28, 28),cmap='gray_r')
    ax.set_title('%d' %y_selected[i], fontsize=16)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### 예측함수

In [ ]:
# 시그모이드 함수
def sigmoid(x):
    return 1/(1+ np.exp(-x))

In [ ]:
# 소프트맥스 함수
def softmax(x):
    x = x.T
    x_max = x.max(axis=0)
    x = x - x_max
    w = np.exp(x)
    return (w / w.sum(axis=0)).T

### 검증함수

In [ ]:
# 교차 엔트로피 함수
def cross_entropy(yt, yp):
    return -np.mean(np.sum(yt * np.log(yp), axis=1))

In [ ]:
# 검증 처리(리턴 값은 정확도와 손실함수)
from sklearn.metrics import accuracy_score

def evaluate(x_test, y_test, y_test_one, V, W):
    b1_test = np.insert(sigmoid(x_test @ V), 0, 1, axis=1)
    yp_test_one = softmax(b1_test @ W)
    yp_test = np.argmax(yp_test_one, axis=1)
    loss = cross_entropy(y_test_one, yp_test_one)
    score = accuracy_score(y_test, yp_test)
    return score, loss   

### 미니 배치 처리

In [ ]:
# 미니 배치용 인덱스 취득 함수
import numpy as np

class Indexes():
    
    # 생성자
    def __init__(self, total, size):
        # 배열 전체의 크기
        self.total   = total
        # 배치 크기
        self.size    = size
        # 작업용 인덱스 초기값은 NULL로 한다.
        self.indexes = np.zeros(0) 

    # 인덱스 취득 함수   
    def next_index(self):
        next_flag = False
        
    # 배치 크기보다 작업용 인덱스가 작은 경우 인덱스를 재생성
        if len(self.indexes) < self.size: 
            self.indexes = np.random.choice(self.total, 
                self.total, replace=False)
            next_flag = True
            
        # 복귀용 인덱스 취득과 작업용 인덱스 갱신
        index = self.indexes[:self.size]
        self.indexes = self.indexes[self.size:]
        return index, next_flag

In [ ]:
# 인덱스 클래스의 테스트

# 클래스 초기화
# 20: 전체 배열의 크기
# 5: 한번에 가져오는 인덱스 수
indexes = Indexes(20, 5)

for i in range(6):
    # next_index 함수 호출
    # 리턴 값1: 인덱스의 numpy 배열
    # 리턴 값2: 작업용 인덱스가 갱신되었는지의 여부
    arr, flag = indexes.next_index()
    print(arr, flag)

### 초기화 처리 1

In [ ]:
# 변수 초기 선언 

# 은닉층의 노드 개수
H = 128
H1 = H + 1
# M: 학습용 데이터 계열의 전체 개수
M  = x_train.shape[0]
# D: 입력 데이터의 차원 수
D = x_train.shape[1]
# N: 분류 클래스의 개수
N = y_train_one.shape[1]

# 반복 횟수
nb_epoch = 100
# 미니 배치 크기
batch_size = 512
B = batch_size
# 학습률
alpha = 0.01

# 가중치 행렬의 초기 설정(모든 값이 1)
V = np.ones((D, H))
W = np.ones((H1, N))

# 검증 결과 기록(손실함수와 정확도)
history1 = np.zeros((0, 3))

# 미니 배치를 위한 초기화
indexes = Indexes(M, batch_size)

# 반복 횟수 카운터 초기화
epoch = 0

### 주요 처리 1

In [ ]:
# 주요 처리
while epoch < nb_epoch:

    # 학습대상 선택(미니 배치 학습법)
    index, next_flag = indexes.next_index()
    x, yt = x_train[index], y_train_one[index]
    
    # 예측값 계산(순전파)
    a = x @ V                         # (10.6.3)
    b = sigmoid(a)                    # (10.6.4)
    b1 = np.insert(b, 0, 1, axis=1)   # 더미 변수의 추가 
    u = b1 @ W                        # (10.6.5)   
    yp = softmax(u)                   # (10.6.6)
    
    # 오차 계산
    yd = yp - yt                      # (10.6.7)
    bd = b * (1-b) * (yd @ W[1:].T)   # (10.6.8)

    # 기울기 계산
    W = W - alpha * (b1.T @ yd) / B   # (10.6.9)
    V = V - alpha * (x.T @ bd) / B    # (10.6.10)
    
    # 로그 기록
    if next_flag: # 1 epoch 종료 후의 처리
        score, loss = evaluate(
            x_test, y_test, y_test_one, V, W)
        history1 = np.vstack((history1, 
            np.array([epoch, loss, score])))
        print("epoch = %d loss = %f score = %f" 
            % (epoch, loss, score))
        epoch = epoch + 1

### 결과 확인 1

In [ ]:
#손실함수와 정확도의 확인
print('[초기 상태] 손실함수: %f, 정확도: %f'
        % (history1[0,1], history1[0,2]))
print('[최종 상태] 손실함수: %f, 정확도: %f'
        % (history1[-1,1], history1[-1,2]))

In [ ]:
# 학습 곡선 표시(손실함수의 값)
plt.plot(history1[:,0], history1[:,1])
plt.ylim(0,2.5)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

In [ ]:
# 학습 곡선 표시(정확도)
plt.plot(history1[:,0], history1[:,2])
plt.ylim(0,1)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

## 10.8 프로그램 구현(2) 매개변수 초기값의 설정 변경

### 초기화 처리  2

In [ ]:
# 변수 초기 선언 가중치 행렬의 초기화 방식 변경

# 은닉층의 노드 개수
H = 128
H1 = H + 1

# M: 학습용 데이터 계열의 전체 개수
M  = x_train.shape[0]

# D: 입력 데이터의 차원 수
D = x_train.shape[1]

# N: 분류 클래스의 개수
N = y_train_one.shape[1]

# 기계 학습 매개변수
alpha = 0.01
nb_epoch = 100
batch_size = 512
B = batch_size

# 가중치 행렬의 초기 설정(모든 값이 1)
V = np.ones((D, H))
W = np.ones((H1, N))

# 검증 결과 기록(손실함수와 정확도)
history2 = np.zeros((0, 3))

# 미니 배치를 위한 초기화
indexes = Indexes(M, batch_size)

# 반복 횟수 카운터 초기화
epoch = 0

In [ ]:
# 개선된 가중치 행렬의 초기화
V = np.random.randn(D, H) / np.sqrt(D / 2)
W = np.random.randn(H1, N) / np.sqrt(H1 / 2)
print(V[:2,:5])
print(W[:2,:5])

### 주요 처리 2

In [ ]:
# 주요 처리
while epoch < nb_epoch:
    
    # 학습대상 선택(미니 배치 학습법)
    index, next_flag = indexes.next_index()
    x, yt = x_train[index], y_train_one[index]

    # 예측값 계산(순전파)
    a = x @ V                         # (10.6.3)
    b = sigmoid(a)                    # (10.6.4)
    b1 = np.insert(b, 0, 1, axis=1)   # 더미 변수의 추가
    u = b1 @ W                        # (10.6.5)
    yp = softmax(u)                   # (10.6.6)
    
    # 오차 계산
    yd = yp - yt                      # (10.6.7)
    bd = b * (1-b) * (yd @ W[1:].T)   # (10.6.8)

    # 기울기 계산
    W = W - alpha * (b1.T @ yd) / B   # (10.6.9)
    V = V - alpha * (x.T @ bd) / B    # (10.6.10)

    if next_flag: # 1epoch 종료 후의 처리
        score, loss = evaluate(
            x_test, y_test, y_test_one, V, W)
        history2 = np.vstack((history2, 
            np.array([epoch, loss, score])))
        print("epoch = %d loss = %f score = %f" 
            % (epoch, loss, score))
        epoch = epoch + 1

### 결과 확인 2

In [ ]:
# 손실함수와 정확도의 확인
print('[초기 상태] 손실함수: %f, 정확도: %f'
        % (history2[0,1], history2[0,2]))
print('[최종 상태] 손실함수: %f, 정확도: %f'
        % (history2[-1,1], history2[-1,2]))

In [ ]:
# 학습 곡선 표시(손실함수의 값)
plt.plot(history2[:,0], history2[:,1])
plt.ylim(0,2.5)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

In [ ]:
# 학습 곡선 표시(정확도)
plt.plot(history2[:,0], history2[:,2])
plt.ylim(0,1)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

## 10.9 프로그램 구현(3) ReLu 함수의 도입

In [ ]:
# ReLU 함수
def ReLU(x):
    return np.maximum(0, x)

In [ ]:
# step 함수
def step(x):
    return 1.0 * ( x > 0)

In [ ]:
# ReLU 함수와 step 함수의 그래프 표시

xx =  np.linspace(-4, 4, 501)
yy = ReLU(xx)
plt.figure(figsize=(6,6))
#plt.ylim(0.0, 1.0)
plt.xlim(-4, 4)
plt.ylim(-4, 4)
plt.xlabel(r'$x$', fontsize=14)
plt.ylabel(r'$y$', fontsize=14)
plt.grid(lw=2)
plt.plot(xx, ReLU(xx), c='b', label='ReLU', linestyle='-', lw=3)
plt.plot(xx, step(xx), c='k', label='step', linestyle='-.', lw=3)
plt.xticks(size=14)
plt.yticks(size=14)
plt.legend(fontsize=14)
plt.show()

### 검증 처리 3

In [ ]:
# 검증 처리 (ReLU 함수 지원)
from sklearn.metrics import accuracy_score

def evaluate2(x_test, y_test, y_test_one, V, W):
    b1_test = np.insert(ReLU(x_test @ V), 0, 1, axis=1)
    yp_test_one = softmax(b1_test @ W)
    yp_test = np.argmax(yp_test_one, axis=1)
    loss = cross_entropy(y_test_one, yp_test_one)
    score = accuracy_score(y_test, yp_test)
    return score, loss   

### 초기화 처리 3

In [ ]:
# 변수 초기 선언 가중치 행렬의 초기화 방식 변경
# 은닉층의 노드 개수
H = 128
H1 = H + 1
# M: 학습용 데이터 계열의 전체 개수
M  = x_train.shape[0]

# D: 입력 데이터의 차원 수
D = x_train.shape[1]

# N: 분류 클래스의 개수
N = y_train_one.shape[1]

# 기계 학습 매개변수
alpha = 0.01
nb_epoch = 100
batch_size = 512
B = batch_size

# 가중치 행렬의 초기 설정
V = np.random.randn(D, H) / np.sqrt(D / 2)
W = np.random.randn(H1, N) / np.sqrt(H1 / 2)

# 검증 결과 기록(손실함수와 정확도)
history3 = np.zeros((0, 3))

# 미니 배치를 위한 초기화
indexes = Indexes(M, batch_size)

# 반복 횟수 카운터 초기화
epoch = 0

### 주요 처리 3

In [ ]:
# 주요 처리 (시그모이드 함수를 ReLU 함수로 변경)
while epoch < nb_epoch:
    
    # 학습대상 선택(미니 배치 학습법)
    index, next_flag = indexes.next_index()
    x, yt = x_train[index], y_train_one[index]

    # 예측값 계산(순전파)
    a = x @ V                         # (10.6.3)
    b = ReLU(a)                       # (10.6.4) ReLU화
    b1 = np.insert(b, 0, 1, axis=1)   # 더미 변수의 추가 
    u = b1 @ W                        # (10.6.5)   
    yp = softmax(u)                   # (10.6.6)
    
    # 오차 계산
    yd = yp - yt                      # (10.6.7)
    bd = step(a) * (yd @ W[1:].T)     #(10.6.8) ReLU화

    # 기울기 계산
    W = W - alpha * (b1.T @ yd) / B   # (10.6.9)
    V = V - alpha * (x.T @ bd) / B    # (10.6.10)

    if next_flag: # 1epoch 종료 후의 처리
        score, loss = evaluate2(
            x_test, y_test, y_test_one, V, W)
        history3 = np.vstack((history3, 
            np.array([epoch, loss, score])))
        print("epoch = %d loss = %f score = %f" 
            % (epoch, loss, score))
        epoch = epoch + 1

### 결과 확인 3

In [ ]:
# 손실함수와 정확도의 확인
print('[초기 상태] 손실함수: %f, 정확도: %f'
        % (history3[0,1], history3[0,2]))
print('[최종 상태] 손실함수: %f, 정확도: %f'
        % (history3[-1,1], history3[-1,2]))

In [ ]:
# 학습 곡선 표시(손실함수의 값)
plt.plot(history3[:,0], history3[:,1])
plt.ylim(0,2.5)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

In [ ]:
# 학습 곡선 표시(정확도)
plt.plot(history3[:,0], history3[:,2])
plt.ylim(0,1)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

In [ ]:
# 데이터 내용 확인
import matplotlib.pyplot as plt
N = 20
np.random.seed(123)
indexes = np.random.choice(y_test.shape[0], N, replace=False)

# x_org의 선택 결과 표시(흑백 반전)
x_selected = x_test[indexes]
y_selected = y_test[indexes]

# 예측값 계산
b1_test = np.insert(ReLU(x_selected @ V), 0, 1, axis=1)
yp_test_one = softmax(b1_test @ W)
yp_test = np.argmax(yp_test_one, axis=1)

# 그래프 표시
plt.figure(figsize=(10, 3))
for i in range(N):
    ax = plt.subplot(2, N/2, i + 1)
    plt.imshow(x_selected[i,1:].reshape(28, 28),cmap='gray_r')
    ax.set_title('%d:%d' % (y_selected[i], yp_test[i]),fontsize=14 )
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 10.10 프로그램 구현(4) 은닉층의 2계층화

### 검증 처리 4

In [ ]:
# 검증 처리 (은닉층에 2계층 지원)
from sklearn.metrics import accuracy_score

def evaluate3(x_test, y_test, y_test_one, U, V, W):
    b1_test = np.insert(ReLU(x_test @ U), 0, 1, axis=1)
    d1_test = np.insert(ReLU(b1_test @ V), 0, 1, axis=1)
    yp_test_one = softmax(d1_test @ W)
    yp_test = np.argmax(yp_test_one, axis=1)
    loss = cross_entropy(y_test_one, yp_test_one)
    score = accuracy_score(y_test, yp_test)
    return score, loss   

### 초기화 처리 4

In [ ]:
# 변수 초기 선언 가중치 행렬의 초기화 방식 변경
# 은닉층의 노드 개수
H = 128
H1 = H + 1
# M: 학습용 데이터 계열의 전체 개수
M  = x_train.shape[0]

# D: 입력 데이터의 차원 수
D = x_train.shape[1]

# N: 분류 클래스의 개수
N = y_train_one.shape[1]

# 기계 학습 매개변수
alpha = 0.01
nb_epoch = 200
batch_size = 512
B = batch_size

# 가중치 행렬의 초기 설정
U = np.random.randn(D, H) / np.sqrt(D / 2)
V = np.random.randn(H1, H) / np.sqrt(H1 / 2)
W = np.random.randn(H1, N) / np.sqrt(H1 / 2)

# 검증 결과 기록(손실함수와 정확도)
history4 = np.zeros((0, 3))

# 미니 배치를 위한 초기화
indexes = Indexes(M, batch_size)

# 반복 횟수 카운터 초기화
epoch = 0

### 주요 처리 4

In [ ]:
# 주요 처리 (은닉층 2계층화)

while epoch < nb_epoch:
    # 학습대상 선택(미니 배치 학습법)
    index, next_flag = indexes.next_index()
    x, yt = x_train[index], y_train_one[index]
    
    # 예측값 계산(순전파)
    a = x @ U                        # (10.6.11)
    b = ReLU(a)                      # (10.6.12)
    b1 = np.insert(b, 0, 1, axis=1)  # 더미 변수의 추가
    c = b1 @ V                       # (10.6.13)
    d = ReLU(c)                      # (10.6.14)
    d1 = np.insert(d, 0, 1, axis=1)  # 더미 변수의 추가
    u = d1 @ W                       # (10.6.15)
    yp = softmax(u)                  # (10.6.16)
    
    # 오차 계산
    yd = yp - yt                     # (10.6.17)
    dd = step(c) * (yd @ W[1:].T)    # (10.6.18)
    bd = step(a) * (dd @ V[1:].T)    # (10.6.19)

    # 기울기 계산
    W = W - alpha * (d1.T @ yd) / B  # (10.6.20)
    V = V - alpha * (b1.T @ dd) / B  # (10.6.21)
    U = U - alpha * (x.T @ bd) / B   # (10.6.22)

    if next_flag: # 1epoch 종료 후의 처리
        score, loss = evaluate3(
            x_test, y_test, y_test_one, U, V, W)
        history4 = np.vstack((history4, 
            np.array([epoch, loss, score])))
        print("epoch = %d loss = %f score = %f" 
            % (epoch, loss, score))
        epoch = epoch + 1

### 결과 확인 4

In [ ]:
# 손실함수와 정확도의 확인
print('[초기 상태] 손실함수: %f, 정확도: %f'
    % (history4[1,1], history4[1,2]))
print('[최종 상태] 손실함수: %f, 정확도: %f'
    % (history4[-1,1], history4[-1,2]))

In [ ]:
# 학습 곡선 표시(손실함수의 값)
plt.plot(history4[:,0], history4[:,1])
plt.ylim(0,2.5)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

In [ ]:
# 학습 곡선 표시(정확도)
plt.plot(history4[:,0], history4[:,2])
plt.ylim(0,1)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

In [ ]:
# 데이터 내용 확인
import matplotlib.pyplot as plt
N = 20
np.random.seed(123)
indexes = np.random.choice(y_test.shape[0], N, replace=False)

# x_org의 선택 결과 표시(흑백 반전)
x_selected = x_test[indexes]
y_selected = y_test[indexes]

# 예측값 계산
b1_test = np.insert(ReLU(x_selected @ U), 0, 1, axis=1)
d1_test = np.insert(ReLU(b1_test @ V), 0, 1, axis=1)
yp_test_one = softmax(d1_test @ W)
yp_test = np.argmax(yp_test_one, axis=1)

# 그래프 표시
plt.figure(figsize=(10, 3))
for i in range(N):
    ax = plt.subplot(2, N/2, i + 1)
    plt.imshow(x_selected[i,1:].reshape(28, 28),cmap='gray_r')
    ax.set_title('%d:%d' % (y_selected[i], yp_test[i]),fontsize=14 )
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()